Script de carga de datos

In [7]:
# Importaciones y configuración inicial
import json
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.chains import RetrievalQA

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Configurar para usar GitHub Models
github_token = os.getenv('GITHUB_TOKEN')
openai_base_url = os.getenv('OPENAI_BASE_URL')

if not github_token:
    print("❌ Error: GITHUB_TOKEN no está configurado")
    print("Asegúrate de tener tu GitHub token en el archivo .env")
else:
    print("✅ GitHub token configurado correctamente")

✅ GitHub token configurado correctamente


In [9]:
# Carga de datos y conversión a documentos
file_path = os.path.join("data", "productos_unimarc_muestra.json")

if not os.path.exists(file_path):
    print(f"❌ Error: No se encontró el archivo {file_path}")
    print("Archivos disponibles en data/:", os.listdir("data"))
    print("💡 Ejecuta primero el script crear_muestra_productos.py para generar la muestra")
else:
    with open(file_path, "r", encoding="utf-8") as f:
        productos = json.load(f)
    
    print(f"✅ Se cargaron {len(productos)} productos de la muestra diversa")
    
    # Mostrar distribución por categoría
    categorias = {}
    for p in productos:
        cat = p['categoria']
        categorias[cat] = categorias.get(cat, 0) + 1
    
    print("📊 Categorías incluidas:")
    for categoria, cantidad in sorted(categorias.items()):
        print(f"  - {categoria}: {cantidad} productos")

    # Convertir productos a documentos de LangChain
    docs = []
    for p in productos:
        contenido = (
            f"Nombre: {p['nombre']} | "
            f"Precio: ${p['precio']} | "
            f"Categoría: {p['categoria']} | "
            f"Subcategoría: {p.get('subcategoria', 'N/A')} | "
            f"Supermercado: {p['supermercado']}"
        )
        docs.append(Document(
            page_content=contenido, 
            metadata={
                "supermercado": p["supermercado"],
                "precio": p["precio"],
                "categoria": p["categoria"],
                "subcategoria": p.get("subcategoria", "N/A"),
                "enlace": p.get("enlace", ""),
                "imagen_url": p.get("imagen_url", "")
            }
        ))

    print(f"✅ Se crearon {len(docs)} documentos")

✅ Se cargaron 495 productos de la muestra diversa
📊 Categorías incluidas:
  - bebidas-y-licores: 55 productos
  - carnes: 55 productos
  - congelados: 55 productos
  - desayuno-y-dulces: 55 productos
  - despensa: 55 productos
  - frutas-y-verduras: 55 productos
  - lacteos-huevos-y-refrigerados: 55 productos
  - panaderia-y-pasteleria: 55 productos
  - quesos-y-fiambres: 55 productos
✅ Se crearon 495 documentos


In [10]:
# Configuración del sistema RAG (embeddings, vectorstore y LLM)
try:
    # Configurar embeddings para GitHub Models
    embeddings = OpenAIEmbeddings(
        api_key=github_token,
        base_url=openai_base_url,
        model="text-embedding-3-small"
    )
    
    print("✅ Creando embeddings...")
    vectorstore = FAISS.from_documents(docs, embeddings)
    print("✅ Vector store creado exitosamente")

    # Crear retriever y LLM usando GitHub Models
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    
    llm = ChatOpenAI(
        api_key=github_token,
        base_url=openai_base_url,
        model="gpt-4o-mini",
        temperature=0
    )
    print("✅ LLM configurado correctamente")

    # Construir pipeline RAG
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff"
    )
    print("✅ Sistema RAG configurado exitosamente")
    
except Exception as e:
    print(f"❌ Error al configurar GitHub Models: {e}")
    print("Tipo de error:", type(e).__name__)
    if "413" in str(e) or "tokens_limit_reached" in str(e):
        print("💡 Sugerencia: El archivo de muestra sigue siendo muy grande, considera reducir más productos")

✅ Creando embeddings...
✅ Vector store creado exitosamente
✅ LLM configurado correctamente
✅ Sistema RAG configurado exitosamente
✅ Vector store creado exitosamente
✅ LLM configurado correctamente
✅ Sistema RAG configurado exitosamente


In [11]:
# Consulta al sistema RAG
consulta = "Arma un carro de compras saludable para una familia de 3 personas con 20 productos de Unimarc. Incluye frutas, verduras, proteínas magras, lácteos bajos en grasa, cereales integrales y snacks saludables. Prioriza productos nutritivos y de buen precio."

print("🔍 Consulta:", consulta)
print("🤖 Procesando respuesta...")

respuesta = qa.run(consulta)
print("🤖 Respuesta:", respuesta)

🔍 Consulta: Arma un carro de compras saludable para una familia de 3 personas con 20 productos de Unimarc. Incluye frutas, verduras, proteínas magras, lácteos bajos en grasa, cereales integrales y snacks saludables. Prioriza productos nutritivos y de buen precio.
🤖 Procesando respuesta...
🤖 Respuesta: Aquí tienes una sugerencia para un carro de compras saludable para una familia de 3 personas con 20 productos de Unimarc, priorizando productos nutritivos y de buen precio:

1. Ensalada genial apio zanahoria bolsa 300 g - $2490
2. Ensalada zanahoria proverde 180 g - $1290
3. Arvejas verdes nuestra cocina partidas bolsa 500 g - $1250
4. Pack barra cereal nature valley avena miel caja 6 un de 42 g - $5050
5. Pack barra cereal nature valley almendra 6 un de 42 g - $5050
6. Yogur natural bajo en grasa (puedes elegir la cantidad que necesites)
7. Pechuga de pollo (puedes elegir la cantidad que necesites)
8. Pescado (puedes elegir la cantidad que necesites)
9. Lentejas (puedes elegir la cantida